In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install --upgrade tensorflow tensorflow-addons pandas matplotlib

In [ ]:
!rm -rf /content/drive/MyDrive/PokemonGenFinal
!unzip /content/drive/MyDrive/PokemonGenFinal.zip -d /content/drive/MyDrive/

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

# Load data from PokemonDataset.csv
csv_file_path = "/content/drive/MyDrive/PokemonGenFinal/pokemon_images.csv"  # Ensure the file path is correct
data_df = pd.read_csv(csv_file_path)

# Add prefix to the image_path column
base_path = "/content/drive/MyDrive/PokemonGenFinal/"
data_df['image_path'] = base_path + data_df['image_path'].astype(str)

# Encode labels (name column)
class_labels = data_df['name'].unique()  # Get unique names as class labels
label_to_index = {label: idx for idx, label in enumerate(class_labels)}  # Create a mapping
data_df['label'] = data_df['name'].map(label_to_index)  # Map names to numerical labels

# Filter out classes with only one instance
data_df = data_df[data_df['label'].map(data_df['label'].value_counts()) > 1]

# Train-test split
train_df, val_df = train_test_split(data_df, test_size=0.2, stratify=data_df['label'])

# Preprocessing function
IMG_SIZE = 224  # MobileNetV3Large input size is 224x224
def preprocess_image(image_path, label):
    image = tf.io.read_file(image_path)  # Read the image file
    image = tf.image.decode_jpeg(image, channels=3)  # Decode the image
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])  # Resize to IMG_SIZE
    image = tf.keras.applications.mobilenet_v3.preprocess_input(image)  # Preprocess for MobileNetV3
    return image, label

# Create TensorFlow datasets
def create_dataset(df):
    file_paths = df['image_path'].values  # Extract file paths
    labels = df['label'].values  # Extract labels
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))  # Create a TensorFlow dataset
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)  # Preprocess images
    dataset = dataset.shuffle(buffer_size=1000).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)  # Optimize dataset
    return dataset

# Create train and validation datasets
train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df)

# Print a summary
print(f"Number of training samples: {len(train_df)}")
print(f"Number of validation samples: {len(val_df)}")
print(f"Class to label mapping: {label_to_index}")

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import MobileNetV3Large

# Define the number of classes
num_classes = len(class_labels)

# Load pre-trained MobileNetV3 with frozen weights
base_model = MobileNetV3Large(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),  # Change shape based on your data
    include_top=False,  # Exclude classifier head
    weights='imagenet'  # Pre-trained on ImageNet
)
base_model.trainable = False  # Freeze the base model initially

# Build the model
model = models.Sequential([
    # Data Augmentation Layers
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(0.1, 0.1),

    # Pre-trained backbone
    base_model,

    # Bottleneck block with correct BatchNorm-Activation order
    layers.Conv2D(256, (1, 1), padding='same', use_bias=False),  # No activation here
    layers.BatchNormalization(),
    layers.Activation('relu'),  # Relu activation after BatchNorm

    layers.DepthwiseConv2D((3, 3), padding='same', use_bias=False),
    layers.BatchNormalization(),
    layers.Activation('relu'),

    layers.Conv2D(512, (1, 1), padding='same', use_bias=False),
    layers.BatchNormalization(),
    layers.Activation('relu'),

    # Global pooling
    layers.GlobalAveragePooling2D(),

    # Dense block with dropout and regularization
    layers.Dense(768, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.3),  # Increased dropout for stronger regularization

    # Output layer
    layers.Dense(num_classes, activation='softmax')  # Adjust num_classes as needed
])

for i, layer in enumerate(model.layers):
    print(f"Layer {i}: {layer.name}, Trainable: {layer.trainable}")

In [ ]:
# Phase 1: Initial Training
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=37
)

In [ ]:
# Save the model in the Keras format
model.save("/content/drive/MyDrive/pre_train_mobile.keras")
print("Model saved successfully in Keras format!")